In [ ]:
reset

In [34]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import csv, glob, os, sys
from sklearn import svm

plt.rcParams['xtick.labelsize'] = 15; plt.rcParams['ytick.labelsize'] = 15 #make size of axis tick labels larger

filename = 'day4_run2_samples9_points150.csv' # designate filename
rawdf = pd.read_table(filename, error_bad_lines=False, delimiter = ',') #read in file
print(rawdf)

      Seconds  Sample    MQ2    MQ4    MQ5    MQ7  Day  Hop  Alpha  Beta  \
0      1168.8       1  0.770  0.550  0.300  0.905    4    1   12.3   3.6   
1      1169.6       1  0.770  0.545  0.295  0.905    4    1   12.3   3.6   
2      1170.4       1  0.770  0.550  0.295  0.905    4    1   12.3   3.6   
3      1171.2       1  0.770  0.550  0.295  0.915    4    1   12.3   3.6   
4      1172.0       1  0.770  0.545  0.300  0.910    4    1   12.3   3.6   
5      1172.9       1  0.770  0.550  0.300  0.915    4    1   12.3   3.6   
6      1173.7       1  0.770  0.555  0.295  0.915    4    1   12.3   3.6   
7      1174.5       1  0.770  0.555  0.300  0.915    4    1   12.3   3.6   
8      1175.3       1  0.770  0.550  0.295  0.915    4    1   12.3   3.6   
9      1176.1       1  0.770  0.555  0.295  0.915    4    1   12.3   3.6   
10     1176.9       1  0.770  0.550  0.300  0.915    4    1   12.3   3.6   
11     1177.7       1  0.775  0.550  0.295  0.915    4    1   12.3   3.6   
12     1178.

In [38]:
labels = np.array(rawdf['Hop']) #set classes as labels

#df = rawdf.filter(['mq2', 'mq4', 'mq5', 'mq7'], axis=1)
#sensordata = df.values #matrix of sensor data

rawdf['mq2/mq7'] = np.divide(np.array(rawdf['MQ2']),np.array(rawdf['MQ7']))
rawdf['mq4/mq7'] = np.divide(np.array(rawdf['MQ4']),np.array(rawdf['MQ7']))
rawdf['mq5/mq7'] = np.divide(np.array(rawdf['MQ5']),np.array(rawdf['MQ7']))

mq2 = np.array(rawdf['MQ2'])
mq5 = np.array(rawdf['MQ5'])
mq7 = np.array(rawdf['MQ7'])

#sensordata = rawdf.filter(['mq2', 'mq4', 'mq5', 'mq7'], axis=1).values #matrix of sensor data
#sensordata = rawdf.filter(['mq2', 'mq5', 'mq7'], axis=1).values #matrix of sensor data

sensordata = [mq2, mq5, mq7]
X = sensordata
y = labels

target_names = ['citra', 'cascade', 'centennial', 'simcoe', 'columbus', 'amarillo', 
                'liberty', 'magnum', 'willamette']

pca = PCA(n_components=3)
X_r = pca.fit(X).transform(X)

# Percentage of variance explained for each component
print('explained variance ratio (PCA): %s'
      % str(pca.explained_variance_ratio_))
#print('PCA log-likelihood = ' + format(pca.score(X,y)))
'''
plt.figure()
colors = ['black', 'cyan', 'darkorange', 'red', 'blue', 'lightblue', 'gray', 'magenta', 'yellow']
lw = .2

for color, i, target_name in zip(colors, [1, 2, 3, 4, 5, 6, 7, 8, 9], target_names):
    plt.scatter(X_r[y == i, 0], X_r[y == i, 1], color=color, alpha=.5, lw=lw,
                label=target_name, s=20)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('PCA', fontsize=15)
plt.show()
'''
pca.components_

explained variance ratio (PCA): [  9.66271532e-01   3.37284678e-02   1.40976061e-30]


array([[-0.03995108, -0.04031719, -0.04031719, ..., -0.01773204,
        -0.01773204, -0.01805406],
       [-0.03117528, -0.03103906, -0.03103906, ...,  0.05093338,
         0.05093338,  0.05446363],
       [-0.0065078 , -0.98121282,  0.00774381, ...,  0.00300288,
         0.00300288, -0.00110531]])

In [ ]:
-

In [ ]:
def make_meshgrid(x, y, h=.005):
    x_min, x_max = x.min() - 0.01, x.max() + 0.01
    y_min, y_max = y.min() - 0.01, y.max() + 0.01
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy
def plot_contoursbest(clf, xx, yy, **params):
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = plt.contourf(xx, yy, Z, **params)
    return out


bestmodel= svm.SVC(kernel='rbf', gamma=.7, C=1).fit(X, y)
besttitle = 'SVC with RBF kernel'

# Set-up 2x2 grid for plotting.
fig, sub = plt.subplots(1, 1)

X0, X1 = X[:, 0], X[:, 1]
xx, yy = make_meshgrid(X0, X1)

plot_contoursbest(bestmodel, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
plt.scatter(X0, X1, c=y, cmap=plt.cm.coolwarm, s=10, edgecolors='k')
#ax.set_xlim(xx.min(), xx.max())
#ax.set_ylim(yy.min(), yy.max())
#    ax.set_xlabel('Sepal length')
#    ax.set_ylabel('Sepal width')
#ax.set_xticks(())
#ax.set_yticks(())
plt.title(besttitle)
print(besttitle + ' score = ' + format(bestmodel.score(X,y)))
plt.show()

In [ ]:
pca_and_classes = np.ones((len(X),len(X[0])+1)) #make new matrix with coordinates and classes for saving
for i in range(len(pca_and_classes)):
    pca_and_classes[i][:len(X[0])] = X[i][0:len(X[0])]
    pca_and_classes[i][len(X[0])] = y[i]
    
# SAVE RESULTS TO CSV FILE

savefile = "PCA_results.csv"
 
#open CSV file
with open(savefile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for row in pca_and_classes:
        writer.writerow(row)   

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import graphviz

# identify classes and input matrix
labels = np.array(rawdf['hop']) #set classes as labels
#df = rawdf.drop('hop', 1) #remove class numbers from data matrix

rawdf['mq2/mq7'] = np.divide(np.array(rawdf['mq2']),np.array(rawdf['mq7']))
rawdf['mq4/mq7'] = np.divide(np.array(rawdf['mq4']),np.array(rawdf['mq7']))
rawdf['mq5/mq7'] = np.divide(np.array(rawdf['mq5']),np.array(rawdf['mq7']))

#sensordata = rawdf.filter(['mq2', 'mq4', 'mq5', 'mq7'], axis=1).values #matrix of sensor data
sensordata = rawdf.filter(['mq2/mq7', 'mq4/mq7', 'mq5/mq7'], axis=1)#.values #matrix of sensor data

sensor = list(sensordata) #list of sensor names

# Use decision tree and print decision flowchart
X = sensordata
y = labels

clf = tree.DecisionTreeClassifier(max_depth=1).fit(X, y)
print('classification score = ' + format(round(clf.score(X,y),4)))

dot_data = tree.export_graphviz(clf, out_file=None, #organize printed decision tree
                         feature_names=sensor,  
                         class_names=np.unique(labels.astype(str)),  
                         filled=True, rounded=True) 

decisiongraph = graphviz.Source(dot_data)  
decisiongraph



In [1]:
import
plt.scatter(X_r[:,0],X_r[:,1])
plt.show()

NameError: name 'plt' is not defined